# Sandbox

In [1]:
import sys
sys.path.append('../sddip')

In [2]:
#%run ../scripts/create_result_directories.py

In [3]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from gurobipy import *

import sddip.tree as tree
import sddip.storage as storage
import sddip.utils as utils
import sddip.config as config


## Data Processing

In [4]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")

scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

### Bus Data

In [5]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


In [6]:
bus_df.dtypes

bus_i       int64
type        int64
Pd          int64
Qd          int64
Gs          int64
Bs          int64
area        int64
Vm        float64
Va          int64
baseKV      int64
zone        int64
Vmax      float64
Vmin      float64
dtype: object

### Branch Data

In [7]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


### Generator Data

In [8]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [9]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [10]:
scenario_df
# TODO Stochastische Nachfrage für jeden Knoten

,t,n,p,Pd
0,1,1,1.0,350
1,2,1,0.5,300
2,2,2,0.5,200


### Power Transfer Distribution Factor

In [11]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

In [12]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

probs = np.empty(n_stages, dtype=object)
# p_d[t,n]
p_d = np.empty(n_stages, dtype=object)

for t in range(n_stages):
    stage_df = scenario_df.loc[scenario_df["t"] == t+1]
    probs[t] = np.array(stage_df.p)
    p_d[t] = np.array(stage_df.Pd)

########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = np.array([probs[t].dot(p_d[t]) for t in range(n_stages)])

## Recombining Tree

In [13]:
# n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()

# node_params_df = scenario_df.drop(["t", "n"], axis=1)
# node_params_dicts = node_params_df.to_dict("records")

# s_tree = tree.RecombiningTree(n_nodes_per_stage)

# s_tree.params["gc"] = gen_cost_df.c1
# s_tree.params["suc"] = gen_cost_df.startup
# s_tree.params["sdc"] = gen_cost_df.startup
# s_tree.params["pg_min"] = gen_df.Pmin
# s_tree.params["pg_max"] = gen_df.Pmax
# s_tree.params["pl_max"] = branch_df.rateA
# s_tree.params = {k: v.values.tolist() for k,v in s_tree.params.items()}
# s_tree.params["ptdf"] = ptdf
# s_tree.params["n_gens"] = len(s_tree.params["gc"])
# s_tree.params["n_buses"] = ptdf.shape[1]

# for stage in s_tree:
#     for node in stage:
#         params = node_params_dicts.pop(0)
#         node.prob = params.pop("p")
#         node.params = params


# for stage in s_tree:
#     stage.params["ex_Pd"] = np.zeros((1,s_tree.params["n_buses"]))
#     for node in stage:
#         stage.params["ex_Pd"] += node.prob*np.array(node.params["Pd"])

## Deterministic Unit Commitment Problem

In [14]:
det_model = Model("Deterministic Problem")


########################################################################################################################
# Add variables
########################################################################################################################
x = np.empty((n_gens,n_stages), dtype=object)
y = np.empty((n_gens,n_stages), dtype=object)
s_up = np.empty((n_gens,n_stages), dtype=object)
s_down = np.empty((n_gens,n_stages), dtype=object)

for g in range(n_gens):
    for t in range(n_stages):
        x[g,t] = det_model.addVar(vtype = GRB.BINARY, name = "x_%i,%i"%(g+1,t+1))
        y[g,t] = det_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "y_%i,%i"%(g+1,t+1))
        s_up[g,t] = det_model.addVar(vtype = GRB.BINARY, name = "s_up%i,%i"%(g+1,t+1))
        s_down[g,t] = det_model.addVar(vtype = GRB.BINARY, name = "s_down%i,%i"%(g+1,t+1))

det_model.update()

########################################################################################################################
# Add objective
########################################################################################################################
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist()

obj = LinExpr()

for t in range(n_stages):
    obj_vars = y[:,t].tolist() + s_up[:,t].tolist() + s_down[:,t].tolist()
    obj += LinExpr(cost_coeffs, obj_vars)

det_model.setObjective(obj)

########################################################################################################################
# Add constraints
########################################################################################################################
# Balance constraints
det_model.addConstrs((quicksum(y[:,t]) == ex_pd[t] for t in range(n_stages)), "balance")

# Generator constraints
for g in range(n_gens):
    det_model.addConstrs((y[g,t] >= pg_min[g]*x[g,t] for t in range(n_stages)), "gen_min")
    det_model.addConstrs((y[g,t] <= pg_max[g]*x[g,t] for t in range(n_stages)), "gen_max")

# Power flow constraints
for l in range(n_lines):
    det_model.addConstrs((quicksum(ptdf[l,b] * quicksum(y[g,t] for g in gens_at_bus[b]) for b in range(n_buses)) 
        <= pl_max for t in range(n_stages)), "flow(1)")
    det_model.addConstrs((-quicksum(ptdf[l,b] * quicksum(y[g,t] for g in gens_at_bus[b]) for b in range(n_buses)) 
        <= pl_max for t in range(n_stages)), "flow(2)")

# Startup and shutdown constraints
for g in range(n_gens):
    det_model.addConstr((x[g,0] <= s_up[g,0]), "up-down(0.1)")
    det_model.addConstr((x[g,0] <= s_up[g,0] - s_down[g,t]), "up-down(0.2)")
    det_model.addConstrs((x[g,t] - x[g,t-1] <= s_up[g,t]  for t in range(1,n_stages)), "up-down(1)")
    det_model.addConstrs((x[g,t] - x[g,t-1] <= s_up[g,t] - s_down[g,t]  for t in range(1,n_stages)), "up-down(2)")

det_model.update()

det_model.optimize()
det_model.printAttr("X")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-25
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14 rows, 8 columns and 20 nonzeros
Model fingerprint: 0x886e56ee
Variable types: 2 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 1e+06]
Presolve removed 14 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1200 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+03, best bound 1.200000000000e+03, gap 0.0000%

    Variable            X 
-------------------------
       x_1,1            1 
       y_1,1          350 


## SDDiP

### Forward pass

In [15]:
########################################################################################################################
# Forward pass
########################################################################################################################

x = np.empty((n_gens,n_stages), dtype=object)
y = np.empty((n_gens,n_stages), dtype=object)
z = np.empty((n_gens,n_stages), dtype=object)
s_up = np.empty((n_gens,n_stages), dtype=object)
s_down = np.empty((n_gens,n_stages), dtype=object)
cv = np.empty(n_stages, dtype=object)
ys_p = np.empty(n_stages, dtype=object)
ys_n = np.empty(n_stages, dtype=object)

cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist()

penalty = 10000

# Cut variable upper bounds
cv_ub = np.full(n_stages, 10000)

# Cut intercepts and cut gradients
ci = np.array([cv_ub])
cg = np.array([[]])

x_trial = np.zeros(n_gens)

samples = [[0,1], [0,0]]
n_samples = len(samples)
v_opt_k =[]

# Solution storage
solution_storage = storage.SolutionStorage()
i = 0
for k in range(n_samples):
    x_trial = np.zeros(n_gens)
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        fw_model = Model("P{}".format(t))

        #Variables
        for g in range(n_gens):
            x[g,t] = fw_model.addVar(vtype = GRB.BINARY, name = "x_%i,%i"%(g+1,t+1))
            y[g,t] = fw_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "y_%i,%i"%(g+1,t+1))
            z[g,t] = fw_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = 1, name = "y_%i,%i"%(g+1,t+1))
            s_up[g,t] = fw_model.addVar(vtype = GRB.BINARY, name = "s_up_%i,%i"%(g+1,t+1))
            s_down[g,t] = fw_model.addVar(vtype = GRB.BINARY, name = "s_down_%i,%i"%(g+1,t+1))
        
        cv[t] = fw_model.addVar(vtype = GRB.CONTINUOUS, name = "cv_%i"%(t+1))
        ys_p[t] = fw_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "ys_p_%i"%(t+1))
        ys_n[t] = fw_model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "ys_n_%i"%(t+1))

        fw_model.update()

        # Objective
        obj_vars = y[:,t].tolist() + s_up[:,t].tolist() + s_down[:,t].tolist()
        obj = LinExpr(cost_coeffs, obj_vars)
        obj += penalty*(ys_p[t]+ ys_n[t])
        fw_model.setObjective(obj)

        # Balance constraints
        fw_model.addConstr((quicksum(y[:,t]) + ys_p[t] - ys_n[t] == p_d[t][n]), "balance")

        # Generator constraints
        fw_model.addConstrs((y[g,t] >= pg_min[g]*x[g,t] for g in range(n_gens)), "gen_min")
        fw_model.addConstrs((y[g,t] <= pg_max[g]*x[g,t] for g in range(n_gens)), "gen_max")

        # Power flow constraints
        fw_model.addConstrs((quicksum(ptdf[l,b] * quicksum(y[g,t] for g in gens_at_bus[b]) for b in range(n_buses)) 
            <= pl_max for l in range(n_lines)), "flow(1)")
        fw_model.addConstrs((-quicksum(ptdf[l,b] * quicksum(y[g,t] for g in gens_at_bus[b]) for b in range(n_buses)) 
            <= pl_max for l in range(n_lines)), "flow(2)")

        # Startup and shutdown constraints    
        fw_model.addConstrs((x[g,t] - z[g,t] <= s_up[g,t]  for g in range(n_gens)), "up-down(1)")
        fw_model.addConstrs((x[g,t] - z[g,t] <= s_up[g,t] - s_down[g,t]  for g in range(n_gens)), "up-down(2)")

        # TODO Ramp rate constraints

        # Copy constraints
        fw_model.addConstrs((z[g,t] == x_trial[g] for g in range(n_gens)), "copy")

        # Cut constraints
        # TODO Calculate cut gradient and intercept
        fw_model.addConstrs((cv[t] >= intcp + grad.T.dot(x[:,t]) for intcp, grad in zip(ci[0], cg[0])), "cut")


        fw_model.update()

        fw_model.optimize()

        # Store xtik, ytik, ztik, vtik
        y_kt = [y[g,t].x for g in range(n_gens)]
        s_up_kt = [s_up[g,t].x for g in range(n_gens)]
        s_down_kt = [s_down[g,t].x for g in range(n_gens)]
        z_kt = [z[g,t].x for g in range(n_gens)]

        p_opt_kt = np.array(y_kt + s_up_kt + s_down_kt)
        c_obj_kt = np.array(cost_coeffs)
        v_opt_kt = c_obj_kt.dot(p_opt_kt)

        v_opt_k[-1] += v_opt_kt

        x_trial = np.array([x[g,t].x for g in range(n_gens)])

        solution = {
            "x": x_trial,
            "y": y_kt,
            "z": z_kt
        }

        solution_storage.add_solution(i, k, t, solution)

        fw_model.printAttr("X")


########################################################################################################################
# Statistical upper bound
########################################################################################################################
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 8 columns and 14 nonzeros
Model fingerprint: 0xf41d61a4
Variable types: 5 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 1e+06]
Presolve removed 7 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 7.000000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     700.0000000  700.00000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 ava

In [16]:
solutions_df = solution_storage.to_dataframe()
solutions_df

x        y      z
i k t                       
0 0 0  [1.0]  [350.0]  [0.0]
    1  [1.0]  [200.0]  [1.0]
  1 0  [1.0]  [350.0]  [0.0]
    1  [1.0]  [300.0]  [1.0]

### Backward pass

In [17]:
########################################################################################################################
# Backward pass
########################################################################################################################

# Binarization
# x_f = 1
# upper_bound = 2
# precision = 3

# binarizer = utils.Binarizer()

# bin_vars, bin_multipliers = binarizer.binary_expansion(x_f, upper_bound, precision)


